In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
INIT_LR=1e-4
EPOCHS=20
BS=32
DIRECTORY=r"C:\Users\akshaya\OneDrive\Desktop\Face-Mask-Detection-master\Face-Mask-Detection-master"
CATEGORIES=["with_mask","without_mask"]
print("[INFO] loading images...")
data=[]
labels=[]
for category in CATEGORIES:
    path=os.path.join(DIRECTORY,category)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image=load_img(img_path,target_size=(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)
        data.append(image)
        labels.append(category)
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)
data=np.array(data,dtype="float32")
labels=np.array(labels)
(trainX, testX, trainY, testY)=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=42)
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")
basemodel=MobileNetV2(weights="imagenet",include_top=False,input_tensor=input(shape=(224,224,3)))
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation="softmax")(headModel)
model=Model(inputs=baseModel.input,outputs=headModel)
for layer in baseModel.layers:
    layer.trainable=False
print("[INFO] compiling model...")
opt=Adam(Lr=INIT_LR,decay=INIT_LR/EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
print("[INFO] training head...")
H=model.fit(aug.flow(trainX,trainY,batch_size=BS),steps_per_epoch=len(trainX)//BS,validation_data=(testX,testY),validation_steps=len(testX)//BS,epochs=EPOCHS)
print("[INFO] evaluating network...")
predIdxs=model.predict(testX,betch_size=BS)
predIdxs=np.argmax(predIdxs,axis=1)
#serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detector.model",save_format="h5")
#plot the training loss and accuracy
N=EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arrange(0,n),H.history["loss"],label="train_loss")
plt.plot(np.arrange(0,n),H.history["val_loss"],label="val_loss")
plt.plot(np.arrange(0,n),H.history["accuracy"],label="train_acc")
plt.plot(np.arrange(0,n),H.history["val_accuracy"],label="val_acc")
plt.title("training loss and accuracy")
plt.xlabel("Epoch #")
plt.ylabel("loss/accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

[INFO] loading images...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\akshaya\\OneDrive\\Desktop\\Face-Mask-Detection-master\\Face-Mask-Detection-master\\with_mask'